# Data here: 
## One and two day type 
## Three day type

# Day Type: One/two
## Create Date and Day of Week columns in the DF

In [208]:
#importing packages needed for analysis
import os
import numpy as np
import pandas as pd
import math
from pandas import DataFrame

path = os.getcwd()
#print(path)

#this code creates an output directory in the parent director, if one does not exist yet
#Note: this is where all of the output files will be written, since outputs are large this saves space in git
path = os.getcwd()
parent = os.path.dirname(path)
outputs_dir = parent+'\outputs'
if not os.path.exists(outputs_dir):
    os.makedirs(outputs_dir)
print('output files are written out in parent directory: '+outputs_dir)

load_dur = pd.read_csv('../outputs/load_long_format.csv')
solar_dur = pd.read_csv('../outputs/solar_long_format.csv')
wind_dur = pd.read_csv('../outputs/wind_long_format.csv')

## UNCOMMENT WHICH PROFILE TO BE USED
x = load_dur
x_name = 'load'
x_name2 = 'Load'
x_column = 'Load'

#x = solar_dur
#x_name = 'solar'
#x_name2 = 'Solar_Gen'
#choose TRG 
#x_column = 'TRG6'

#x = wind_dur
#x_name = 'wind'
#x_name2 = 'Wind_Gen'
#x_column = 'TRG4'

x = x[['Region','R_Group','R_Subgroup','Season','Month','Day','Hour',x_column]]
years = pd.read_csv('inputs/years.csv').dropna()

#print(x.head())

output files are written out in parent directory: C:\Users\cmarcy\Desktop\py_projects\temporal\temporal_working\outputs


In [188]:
import datetime
daydata = pd.read_csv('inputs/days_365.csv')
#print(daydata.tail())
daydata = daydata.drop(columns='Month')
x2 = pd.merge(x,daydata,on=['Day'],how='left')
#print(x2.tail())

#sets the year for each region
x2['Year']=2011
x2.loc[x2['R_Group'] == 'ERC', 'Year'] = 2016
#print(x2.head())

#Creates a date column
x2 = x2.rename(columns={'Day':'DOY','DayofMo':'Day'})
x2['Date']=pd.to_datetime(x2[['Year', 'Month', 'Day']], errors='coerce')
#print(x2.tail())

#convert date to a datetime type 
#x2['Date'] = pd.to_datetime(x2['Date'])
x2['DOW'] = x2['Date'].dt.weekday

#check if it is a weekday or not 
weekday = pd.read_csv('inputs/weekday.csv')
x2 = pd.merge(x2,weekday,on='DOW',how='left')
print(x2.head())

  Region R_Group R_Subgroup  Season  Month  DOY  Hour   Load  Day  Year  \
0   FRCC    FRCC       FRCC  winter      1    1     1  17005    1  2011   
1   FRCC    FRCC       FRCC  winter      1    2     1  15750    2  2011   
2   FRCC    FRCC       FRCC  winter      1    3     1  16123    3  2011   
3   FRCC    FRCC       FRCC  winter      1    4     1  18300    4  2011   
4   FRCC    FRCC       FRCC  winter      1    5     1  17370    5  2011   

        Date  DOW  Weekday  
0 2011-01-01    5    False  
1 2011-01-02    6    False  
2 2011-01-03    0     True  
3 2011-01-04    1     True  
4 2011-01-05    2     True  


## Case 1: Monthly, single day type, 24 hours (288 segments)
#### Methodology: Using groupby function to group first by month, then by 24 hours

In [189]:
case1_x = x2.copy()

aggregations = {x_column:['count',sum,'mean']}
case1 = case1_x.groupby(['Region','Month','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case1.head())
print('number of rows in dataset =',case1.shape[0])
case1.to_csv('../outputs/'+x_name+'_segments_1daytype_monthly_24hr.csv')
print()

case1_x2 = pd.merge(case1_x,case1,on=['Region','Month','Hour'],how='left')
case1_x2 = case1_x2.sort_values(['Region',x_column]).drop(case1_x2.columns[0], axis=1)
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case1_x2.to_csv('../outputs/'+x_name+'_8760_1daytype_monthly_24hr.csv')

  Region  Month  Hour  Hour_Tot  Load_Tot      Load_Avg
0   FRCC      1     1        31    626801  20219.387097
1   FRCC      1     2        31    644817  20800.548387
2   FRCC      1     3        31    694961  22418.096774
3   FRCC      1     4        31    781019  25194.161290
4   FRCC      1     5        31    836184  26973.677419
number of rows in dataset = 288

    R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
366    FRCC       FRCC    winter      1    2     2  15694    2  2011   
73     FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
1      FRCC       FRCC    winter      1    2     1  15750    2  2011   

          Date  DOW  Weekday  Hour_Tot  Load_Tot      Load_Avg  
366 2011-01-02    6    False        31    644817  20800.548387  
73  2011-03-15    1     True        31    519021  16742.612903  
1   2011-01-02    6    False        31    626801  20219.387097  
number of rows in dataset = 8760


## Case 2: Season, single day-type, 24 hours (72 segments)
#### Methodology: Use groupby function to group by season and hour

In [190]:
case2_x = x2.copy()
case2_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case2_seasons = case2_seasons.drop(['bimonthly'], axis=1)
case2_seasons = case2_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})
#print(case2_seasons)

case2_x = pd.merge(case2_x, case2_seasons, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case2 = case2_x.groupby(['Region','Season_Group','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season_Group','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case2.head())
print('number of rows in dataset =',case2.shape[0])
case2.to_csv('../outputs/'+x_name+'_segments_1daytype_season_24hr.csv')
print()

case2_x2 = pd.merge(case2_x,case2,on=['Region','Season_Group','Hour'],how='left')
case2_x2 = case2_x2.sort_values(['Region',x_column]).drop(case2_x2.columns[0],axis=1)
print(case2_x2.head(3))
print('number of rows in dataset =',case2_x2.shape[0])
case2_x2.to_csv('../outputs/'+x_name+'_8760_1daytype_season_24hr.csv')

  Region  Season_Group  Hour  Hour_Tot  Load_Tot      Load_Avg
0   FRCC             1     1        90   1659148  18434.977778
1   FRCC             1     2        90   1694864  18831.822222
2   FRCC             1     3        90   1817817  20197.966667
3   FRCC             1     4        90   2039870  22665.222222
4   FRCC             1     5        90   2183989  24266.544444
number of rows in dataset = 120

    R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
366    FRCC       FRCC    winter      1    2     2  15694    2  2011   
73     FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
1      FRCC       FRCC    winter      1    2     1  15750    2  2011   

          Date  DOW  Weekday  Season_Group  Hour_Tot  Load_Tot      Load_Avg  
366 2011-01-02    6    False             1        90   1694864  18831.822222  
73  2011-03-15    1     True             2       122   2159594  17701.590164  
1   2011-01-02    6    False             1        90   1659148 

## Case 3: Monthly, weekend/weekday, 24 hours (576 segments)
#### Metholodogy: Use groupby function to group by month, then weekend/weekday, then by 24 hours

In [191]:
case3_x = x2.copy()

aggregations = {x_column:['count',sum,'mean']}
case3 = case3_x.groupby(['Region','Month','Weekday','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Month','Weekday','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case1.head())
print('number of rows in dataset =',case3.shape[0])
case3.to_csv('../outputs/'+x_name+'_segments_2daytype_monthly_24hr.csv')
print()

case3_x2 = pd.merge(case3_x,case3,on=['Region','Month','Weekday','Hour'],how='left')
case3_x2 = case3_x2.sort_values(['Region',x_column]).drop(case3_x2.columns[0],axis=1)
print(case3_x2.head())
print('number of rows in dataset =',case3_x2.shape[0])
case3_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_monthly_24hr.csv')

  Region  Month  Hour  Hour_Tot  Load_Tot      Load_Avg
0   FRCC      1     1        31    626801  20219.387097
1   FRCC      1     2        31    644817  20800.548387
2   FRCC      1     3        31    694961  22418.096774
3   FRCC      1     4        31    781019  25194.161290
4   FRCC      1     5        31    836184  26973.677419
number of rows in dataset = 576

    R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
366    FRCC       FRCC    winter      1    2     2  15694    2  2011   
73     FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
1      FRCC       FRCC    winter      1    2     1  15750    2  2011   
443    FRCC       FRCC  shoulder      3   79     2  15822   20  2011   
72     FRCC       FRCC  shoulder      3   73     1  15831   14  2011   

          Date  DOW  Weekday  Hour_Tot  Load_Tot      Load_Avg  
366 2011-01-02    6    False        10    212142  21214.200000  
73  2011-03-15    1     True        23    382919  16648.652174  
1  

## Case 4: Season, weekend/weekday, 24 hours (144 segments)
#### Methodology: groupby season, weekday, hour

In [192]:
case4_x = x2.copy()
case4_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case4_seasons = case4_seasons.drop(['bimonthly'], axis=1)
case4_seasons = case4_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case4_x = pd.merge(case4_x, case4_seasons, on='Month', how='left')
#print(case4_x)

aggregations = {x_column:['count',sum,'mean']}
case4 = case4_x.groupby(['Region','Season_Group','Weekday','Hour'],as_index=False).agg(aggregations)
case4.columns = case4.columns.droplevel(0)
case4.columns = ['Region','Season_Group','Weekday','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case4.tail())
print('number of rows in dataset =',case4.shape[0])
case4.to_csv('../outputs/'+x_name+'_segments_2daytype_season_24hr.csv')
print()

case4_x2 = pd.merge(case4_x,case4,on=['Region','Season_Group','Weekday','Hour'],how='left')
case4_x2 = case4_x2.sort_values(['Region',x_column]).drop(case4_x2.columns[0],axis=1)
print(case4_x2.head())
print('number of rows in dataset =',case4_x2.shape[0])
case4_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_season_24hr.csv')

    Region  Season_Group  Weekday  Hour  Hour_Tot  Load_Tot      Load_Avg
235   FRCC             5     True    20        44   1435196  32618.090909
236   FRCC             5     True    21        44   1307046  29705.590909
237   FRCC             5     True    22        44   1191436  27078.090909
238   FRCC             5     True    23        44   1111291  25256.613636
239   FRCC             5     True    24        44   1057027  24023.340909
number of rows in dataset = 240

    R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
366    FRCC       FRCC    winter      1    2     2  15694    2  2011   
73     FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
1      FRCC       FRCC    winter      1    2     1  15750    2  2011   
443    FRCC       FRCC  shoulder      3   79     2  15822   20  2011   
72     FRCC       FRCC  shoulder      3   73     1  15831   14  2011   

          Date  DOW  Weekday  Season_Group  Hour_Tot  Load_Tot      Load_Avg  
366 2011-01

## 4 hour interval day types

In [193]:
#read in 4 hour interval counter
interval_4hr = pd.read_csv('inputs/interval_4hr.csv')
#print(interval_4hr)

x3 = pd.merge(x2,interval_4hr,on='Hour',how='left')
print(x3.tail())

     Region R_Group R_Subgroup  Season  Month  DOY  Hour   Load  Day  Year  \
8755   FRCC    FRCC       FRCC  winter     12  361    24  17370   27  2011   
8756   FRCC    FRCC       FRCC  winter     12  362    24  19366   28  2011   
8757   FRCC    FRCC       FRCC  winter     12  363    24  19024   29  2011   
8758   FRCC    FRCC       FRCC  winter     12  364    24  16730   30  2011   
8759   FRCC    FRCC       FRCC  winter     12  365    24  17794   31  2011   

           Date  DOW  Weekday  4-hr  
8755 2011-12-27    1     True     6  
8756 2011-12-28    2     True     6  
8757 2011-12-29    3     True     6  
8758 2011-12-30    4     True     6  
8759 2011-12-31    5    False     6  


## Case 5: Monthly, single day type, 4 hour intervals (72 segments)
#### Methodology: use groupby by month, 4 hour intervals

In [194]:
case5_x = x3.copy()

aggregations = {x_column:['count',sum,'mean']}
case5 = case5_x.groupby(['Region','Month','4-hr'],as_index=False).agg(aggregations)
case5.columns = case5.columns.droplevel(0)
case5.columns = ['Region','Month','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case5.head())
print('number of rows in dataset =',case5.shape[0])
case5.to_csv('../outputs/'+x_name+'_segments_2daytype_month_4hr.csv')
print()

case5_x2 = pd.merge(case5_x,case5,on=['Region','Month','4-hr'],how='left')
case5_x2 = case5_x2.sort_values(['Region',x_column]).drop(case5_x2.columns[0],axis=1)
print(case5_x2.head())
print('number of rows in dataset =',case5_x2.shape[0])
case5_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_month_4hr.csv')

  Region  Month  4-hr  Hour_Tot  Load_Tot      Load_Avg
0   FRCC      1     1       124   2747598  22158.048387
1   FRCC      1     2       124   3359684  27094.225806
2   FRCC      1     3       124   3143101  25347.588710
3   FRCC      1     4       124   3171871  25579.604839
4   FRCC      1     5       124   3273408  26398.451613
number of rows in dataset = 72

    R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
366    FRCC       FRCC    winter      1    2     2  15694    2  2011   
73     FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
1      FRCC       FRCC    winter      1    2     1  15750    2  2011   
443    FRCC       FRCC  shoulder      3   79     2  15822   20  2011   
72     FRCC       FRCC  shoulder      3   73     1  15831   14  2011   

          Date  DOW  Weekday  4-hr  Hour_Tot  Load_Tot      Load_Avg  
366 2011-01-02    6    False     1       124   2747598  22158.048387  
73  2011-03-15    1     True     1       124   2231100  1

## Case 6: Bi-monthly weekend/weekday day-types, 4-hour intervals
#### Methodology: use groupby function and bimonthly groups

In [198]:
case6_x = x3.copy()
case6_bimonth = pd.read_csv('inputs/season_bimonthly.csv')
case6_bimonth = case6_bimonth.drop(['seasonal'], axis=1)
case6_bimonth = case6_bimonth.rename(columns={'bimonthly':'Bimonth','month':'Month'})

case6_x = pd.merge(case6_x, case6_bimonth, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case6 = case6_x.groupby(['Region','Bimonth','Weekday','4-hr'],as_index=False).agg(aggregations)
case6.columns = case6.columns.droplevel(0)
case6.columns = ['Region','Bimonth','Weekday','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case6.head())
print('number of rows in dataset =',case6.shape[0])
case6.to_csv('../outputs/'+x_name+'_segments_2daytype_bimonth_4hr.csv')
print()

case6_x2 = pd.merge(case6_x,case6,on=['Region','Bimonth','Weekday','4-hr'],how='left')
case6_x2 = case6_x2.sort_values(['Region',x_column]).drop(case6_x2.columns[0],axis=1)
print(case6_x2.head())
print('number of rows in dataset =',case6_x2.shape[0])
case6_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_bimonth_4hr.csv')

  Region  Bimonth  Weekday  4-hr  Hour_Tot  Load_Tot      Load_Avg
0   FRCC        1    False     1        72   1488088  20667.888889
1   FRCC        1    False     2        72   1814946  25207.583333
2   FRCC        1    False     3        72   1787406  24825.083333
3   FRCC        1    False     4        72   1785866  24803.694444
4   FRCC        1    False     5        72   1816454  25228.527778
number of rows in dataset = 72

    R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
366    FRCC       FRCC    winter      1    2     2  15694    2  2011   
73     FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
1      FRCC       FRCC    winter      1    2     1  15750    2  2011   
443    FRCC       FRCC  shoulder      3   79     2  15822   20  2011   
72     FRCC       FRCC  shoulder      3   73     1  15831   14  2011   

          Date  DOW  Weekday  4-hr  Bimonth  Hour_Tot  Load_Tot      Load_Avg  
366 2011-01-02    6    False     1        1        72

## Case 7: Season-based months, weekend/weekday day-types, 4-hour intervals
#### Methodology: groupby function and applied season groups

In [199]:
case7_x = x3.copy()
case7_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case7_seasons = case7_seasons.drop(['bimonthly'], axis=1)
case7_seasons = case7_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case7_x = pd.merge(case7_x, case7_seasons, on='Month', how='left')
#print(case4_load)

aggregations = {x_column:['count',sum,'mean']}
case7 = case7_x.groupby(['Region','Season','Weekday','4-hr'],as_index=False).agg(aggregations)
case7.columns = case7.columns.droplevel(0)
case7.columns = ['Region','Season','Weekday','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case7.head())
print('number of rows in dataset =',case7.shape[0])
case7.to_csv('../outputs/'+x_name+'_segments_2daytype_season_wkd_4hr.csv')
print()

case7_x2 = pd.merge(case7_x,case7,on=['Region','Season','Weekday','4-hr'],how='left')
case7_x2 = case7_x2.sort_values(['Region',x_column]).drop(case7_x2.columns[0],axis=1)
print(case7_x2.head())
print('number of rows in dataset =',case7_x2.shape[0])
case7_x2.to_csv('../outputs/'+x_name+'_8760_2daytype_season_wkd_4hr.csv')

  Region    Season  Weekday  4-hr  Hour_Tot  Load_Tot      Load_Avg
0   FRCC  shoulder    False     1       140   2507878  17913.414286
1   FRCC  shoulder    False     2       140   3152037  22514.550000
2   FRCC  shoulder    False     3       140   3846861  27477.578571
3   FRCC  shoulder    False     4       140   4043491  28882.078571
4   FRCC  shoulder    False     5       140   3739162  26708.300000
number of rows in dataset = 36

    R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
366    FRCC       FRCC    winter      1    2     2  15694    2  2011   
73     FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
1      FRCC       FRCC    winter      1    2     1  15750    2  2011   
443    FRCC       FRCC  shoulder      3   79     2  15822   20  2011   
72     FRCC       FRCC  shoulder      3   73     1  15831   14  2011   

          Date  DOW  Weekday  4-hr  Season_Group  Hour_Tot  Load_Tot  \
366 2011-01-02    6    False     1             1       

# Day type: Three (Weekday, Weekend, Peak Load)
## Find Peak Load Days in Each Month

In [209]:
#create temporary DF to find peak
test = load_dur.copy()

#groupby region, month, and day to sum the total day
aggregations1 = {x_column:sum}
md_sum = test.groupby(['Region','Month','Day'],as_index=False).agg(aggregations1)
#test1.columns = test1.columns.droplevel(0)
md_sum.columns = ['Region','Month','Day',x_name2+'_Tot']
#print(test_sum.head())
#print('number of rows in dataset =',test_sum.shape[0])

md_sum2 = pd.merge(test,md_sum,on=['Region','Month','Day'],how='left')
#print(test3)

#groupby region and month to find maximum 
aggregations2 = {x_name2+'_Tot':max}
md_max = md_sum2.groupby(['Region','Month'],as_index=False).agg(aggregations2)
md_max.columns = ['Region','Month',x_name2+'_Max']
print(md_max)
print()

peakd = pd.merge(test3,md_max,on=['Region','Month'],how='left')
print(peakd.tail())

   Region  Month  Load_Max
0    FRCC      1    839413
1    FRCC      2    632009
2    FRCC      3    629063
3    FRCC      4    774706
4    FRCC      5    761238
5    FRCC      6    855573
6    FRCC      7    848154
7    FRCC      8    864191
8    FRCC      9    798533
9    FRCC     10    707155
10   FRCC     11    657292
11   FRCC     12    584382

     Region R_Group R_Subgroup  Season  Month  Day  Hour   Load  Load_Tot  \
8755   FRCC    FRCC       FRCC  winter     12  361    24  17370    556346   
8756   FRCC    FRCC       FRCC  winter     12  362    24  19366    546778   
8757   FRCC    FRCC       FRCC  winter     12  363    24  19024    564643   
8758   FRCC    FRCC       FRCC  winter     12  364    24  16730    542878   
8759   FRCC    FRCC       FRCC  winter     12  365    24  17794    528679   

      Load_Max  
8755    584382  
8756    584382  
8757    584382  
8758    584382  
8759    584382  


In [210]:
x_peak = x2.copy()

x_peak = pd.merge(x_peak,peakd,on=['Region','R_Group','R_Subgroup','Season','Month','Day','Hour',x_column],how='left')
x_peak = x_peak.rename(columns={'Weekday':'Day_Type'})

#Return True if the load total equals the day identified as the max
x_peak.loc[x_peak['Day_Type'] == True, 'Day_Type'] = 'Weekday'
x_peak.loc[x_peak['Day_Type'] == False, 'Day_Type'] = 'Weekend'
x_peak.loc[x_peak[x_name2+'_Tot'] == x_peak[x_name2+'_Max'], 'Day_Type'] = 'Peak'
x_peak = x_peak.drop([x_name2+'_Tot',x_name2+'_Max'], axis=1)
print(x_peak.head())

  Region R_Group R_Subgroup  Season  Month  DOY  Hour   Load  Day  Year  \
0   FRCC    FRCC       FRCC  winter      1    1     1  17005    1  2011   
1   FRCC    FRCC       FRCC  winter      1    2     1  15750    2  2011   
2   FRCC    FRCC       FRCC  winter      1    3     1  16123    3  2011   
3   FRCC    FRCC       FRCC  winter      1    4     1  18300    4  2011   
4   FRCC    FRCC       FRCC  winter      1    5     1  17370    5  2011   

        Date  DOW Day_Type  
0 2011-01-01    5  Weekend  
1 2011-01-02    6  Weekend  
2 2011-01-03    0  Weekday  
3 2011-01-04    1  Weekday  
4 2011-01-05    2  Weekday  


## Case 1: Monthly, Weekend/weekday/peak day-types, 24 hours (864 segments)
#### Methodology: similar to two day type, just adding in peak day types to sort by

In [150]:
case1_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case1 = case1_x.groupby(['Region','Month','Day_Type','Hour'],as_index=False).agg(aggregations)
case1.columns = case1.columns.droplevel(0)
case1.columns = ['Region','Month','Day_Type','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case1.head())
print('number of rows in dataset =',case1.shape[0])
case1.to_csv('../outputs/'+x_name+'_segments_3daytype_monthly_24hr.csv')
print()

case1_x2 = pd.merge(case1_x,case1,on=['Region','Month','Day_Type','Hour'],how='left')
case1_x2 = case1_x2.sort_values(['Region',x_column])
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case1_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_monthly_24hr.csv')

  Region  Month Day_Type  Hour  Hour_Tot  Load_Tot  Load_Avg
0   FRCC      1     Peak     1         1     32350   32350.0
1   FRCC      1     Peak     2         1     33972   33972.0
2   FRCC      1     Peak     3         1     37185   37185.0
3   FRCC      1     Peak     4         1     41916   41916.0
4   FRCC      1     Peak     5         1     44276   44276.0
number of rows in dataset = 600

     Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
300    FRCC    FRCC       FRCC    winter      1    2     2  15694    2  2011   
4034   FRCC    FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
288    FRCC    FRCC       FRCC    winter      1    2     1  15750    2  2011   

           Date  DOW Day_Type  Hour_Counter  Hour_Tot  Load_Tot      Load_Avg  
300  2011-01-02    6  Weekend            26        10    212142  21214.200000  
4034 2011-03-15    1  Weekday           337        23    382919  16648.652174  
288  2011-01-02    6  Weekend           

## Case 2: Season, weekend/weekday/peak day-types, 24-hours (216 segments)

In [151]:
case2_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case2 = case2_x.groupby(['Region','Season','Day_Type','Hour'],as_index=False).agg(aggregations)
case2.columns = case2.columns.droplevel(0)
case2.columns = ['Region','Season','Day_Type','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case2.head())
print('number of rows in dataset =',case2.shape[0])
case2.to_csv('../outputs/'+x_name+'_segments_3daytype_season_24hr.csv')
print()

case2_x2 = pd.merge(case2_x,case2,on=['Region','Season','Day_Type','Hour'],how='left')
case2_x2 = case2_x2.sort_values(['Region',x_column])
print(case1_x2.head(3))
print('number of rows in dataset =',case1_x2.shape[0])
case2_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_season_24hr.csv')

  Region    Season Day_Type  Hour  Hour_Tot  Load_Tot      Load_Avg
0   FRCC  shoulder  Weekday     1        87   1541095  17713.735632
1   FRCC  shoulder  Weekday     2        87   1553709  17858.724138
2   FRCC  shoulder  Weekday     3        87   1661960  19102.988506
3   FRCC  shoulder  Weekday     4        87   1886198  21680.436782
4   FRCC  shoulder  Weekday     5        87   2013602  23144.850575
number of rows in dataset = 168

     Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
300    FRCC    FRCC       FRCC    winter      1    2     2  15694    2  2011   
4034   FRCC    FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
288    FRCC    FRCC       FRCC    winter      1    2     1  15750    2  2011   

           Date  DOW Day_Type  Hour_Counter  Hour_Tot  Load_Tot      Load_Avg  
300  2011-01-02    6  Weekend            26        10    212142  21214.200000  
4034 2011-03-15    1  Weekday           337        23    382919  16648.652174 

## Case 3: Annual, weekend/weekday/peak day-types, 24-hours (72 segments)

In [152]:
case3_x = x_peak.copy()

aggregations = {x_column:['count',sum,'mean']}
case3 = case3_x.groupby(['Region','Day_Type','Hour'],as_index=False).agg(aggregations)
case3.columns = case3.columns.droplevel(0)
case3.columns = ['Region','Day_Type','Hour','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case3.head())
print('number of rows in dataset =',case3.shape[0])
case3.to_csv('../outputs/'+x_name+'_segments_3daytype_annual_24hr.csv')
print()

case3_x2 = pd.merge(case3_x,case3,on=['Region','Day_Type','Hour'],how='left')
case3_x2 = case3_x2.sort_values(['Region',x_column])
print(case3_x2.head(3))
print('number of rows in dataset =',case3_x2.shape[0])
case3_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_annual_24hr.csv')

  Region Day_Type  Hour  Hour_Tot  Load_Tot  Load_Avg
0   FRCC     Peak     1         1     32350   32350.0
1   FRCC     Peak     2         1     33972   33972.0
2   FRCC     Peak     3         1     37185   37185.0
3   FRCC     Peak     4         1     41916   41916.0
4   FRCC     Peak     5         1     44276   44276.0
number of rows in dataset = 72

     Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
300    FRCC    FRCC       FRCC    winter      1    2     2  15694    2  2011   
4034   FRCC    FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
288    FRCC    FRCC       FRCC    winter      1    2     1  15750    2  2011   

           Date  DOW Day_Type  Hour_Counter  Hour_Tot  Load_Tot      Load_Avg  
300  2011-01-02    6  Weekend            26       105   2051738  19540.361905  
4034 2011-03-15    1  Weekday           337       259   5088736  19647.629344  
288  2011-01-02    6  Weekend            25       105   2072468  19737.790476  
num

## 4 hour interval

In [153]:
#read in 4 hour interval counter
interval_4hr = pd.read_csv('inputs/sequential_hours.csv')
interval_4hr = interval_4hr.drop(columns=['2-hr','8-hr','12-hr','24-hr','48-hr','120-hr'])
#print(interval_4hr)

#add an hour counter
x_peak['Hour_Counter'] = (x_peak['Hour']) + (x_peak['Day'] - 1) * 24
x_peak = x_peak.sort_values(by=['Region','Hour_Counter'])
unique_hc = pd.Series(x_peak['Hour_Counter'].unique()).dropna()
#print(unique_hc.tail(2))

x_peak2 = pd.merge(x_peak,interval_4hr,on='Hour_Counter',how='left')

## Case 4: Bi-monthly, weekend/weekday/peak day-types, 4-hour intervals

In [154]:
case4_x = x_peak2.copy()
case4_bimonth = pd.read_csv('inputs/season_bimonthly.csv')
case4_bimonth = case4_bimonth.drop(['seasonal'], axis=1)
case4_bimonth = case4_bimonth.rename(columns={'bimonthly':'Bimonth','month':'Month'})

case4_x = pd.merge(case4_x, case4_bimonth, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case4 = case4_x.groupby(['Region','Bimonth','Day_Type','4-hr'],as_index=False).agg(aggregations)
case4.columns = case4.columns.droplevel(0)
case4.columns = ['Region','Bimonth','Day_Type','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case4.head())
print('number of rows in dataset =',case4.shape[0])
case4.to_csv('../outputs/'+x_name+'_segments_3daytype_bimonth_4hr.csv')
print()

case4_x2 = pd.merge(case4_x,case4,on=['Region','Bimonth','Day_Type','4-hr'],how='left')
case4_x2 = case4_x2.sort_values(['Region',x_column])
print(case4_x2.head())
print('number of rows in dataset =',case4_x2.shape[0])
case4_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_bimonth_4hr.csv')

  Region  Bimonth Day_Type  4-hr  Hour_Tot  Load_Tot  Load_Avg
0   FRCC        1     Peak    73         4    145423  36355.75
1   FRCC        1     Peak    74         4    167231  41807.75
2   FRCC        1     Peak    75         4    128113  32028.25
3   FRCC        1     Peak    76         4    125565  31391.25
4   FRCC        1     Peak    77         4    145710  36427.50
number of rows in dataset = 1728

     Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
300    FRCC    FRCC       FRCC    winter      1    2     2  15694    2  2011   
4034   FRCC    FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
288    FRCC    FRCC       FRCC    winter      1    2     1  15750    2  2011   
5486   FRCC    FRCC       FRCC  shoulder      3   79     2  15822   20  2011   
3746   FRCC    FRCC       FRCC  shoulder      3   73     1  15831   14  2011   

           Date  DOW Day_Type  Hour_Counter  4-hr  Bimonth  Hour_Tot  \
300  2011-01-02    6  Weekend      

## Case 5: Season-based months, weekend/weekday/peak day-types, 4-hour intervals

In [155]:
case5_x = x_peak2.copy()
case5_seasons = pd.read_csv('inputs/season_bimonthly.csv')
case5_seasons = case5_seasons.drop(['bimonthly'], axis=1)
case5_seasons = case5_seasons.rename(columns={'seasonal':'Season_Group','month':'Month'})

case5_x = pd.merge(case5_x, case5_seasons, on='Month', how='left')

aggregations = {x_column:['count',sum,'mean']}
case5 = case5_x.groupby(['Region','Season_Group','Day_Type','4-hr'],as_index=False).agg(aggregations)
case5.columns = case5.columns.droplevel(0)
case5.columns = ['Region','Season_Group','Day_Type','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case5.head())
print('number of rows in dataset =',case5.shape[0])
case5.to_csv('../outputs/'+x_name+'_segments_3daytype_seasonbased_4hr.csv')
print()

case5_x2 = pd.merge(case5_x,case5,on=['Region','Season_Group','Day_Type','4-hr'],how='left')
case5_x2 = case5_x2.sort_values(['Region',x_column])
print(case5_x2.head())
print('number of rows in dataset =',case5_x2.shape[0])
case5_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_seasonbased_4hr.csv')

  Region  Season_Group Day_Type  4-hr  Hour_Tot  Load_Tot  Load_Avg
0   FRCC             1     Peak    73         4    145423  36355.75
1   FRCC             1     Peak    74         4    167231  41807.75
2   FRCC             1     Peak    75         4    128113  32028.25
3   FRCC             1     Peak    76         4    125565  31391.25
4   FRCC             1     Peak    77         4    145710  36427.50
number of rows in dataset = 1434

     Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
300    FRCC    FRCC       FRCC    winter      1    2     2  15694    2  2011   
4034   FRCC    FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
288    FRCC    FRCC       FRCC    winter      1    2     1  15750    2  2011   
5486   FRCC    FRCC       FRCC  shoulder      3   79     2  15822   20  2011   
3746   FRCC    FRCC       FRCC  shoulder      3   73     1  15831   14  2011   

           Date  DOW Day_Type  Hour_Counter  4-hr  Season_Group  Hour_Tot  \


## Case 6: Season, weekend/weekday/peak day-types, 4-hour intervals

In [156]:
case6_x = x_peak2.copy()

aggregations = {x_column:['count',sum,'mean']}
case6 = case6_x.groupby(['Region','Season','Day_Type','4-hr'],as_index=False).agg(aggregations)
case6.columns = case6.columns.droplevel(0)
case6.columns = ['Region','Season','Day_Type','4-hr','Hour_Tot',x_name2+'_Tot',x_name2+'_Avg']
print(case6.head())
print('number of rows in dataset =',case6.shape[0])
case6.to_csv('../outputs/'+x_name+'_segments_3daytype_season_4hr.csv')
print()

case6_x2 = pd.merge(case6_x,case6,on=['Region','Season','Day_Type','4-hr'],how='left')
case6_x2 = case6_x2.sort_values(['Region',x_column])
print(case6_x2.head())
print('number of rows in dataset =',case6_x2.shape[0])
case6_x2.to_csv('../outputs/'+x_name+'_8760_3daytype_season_4hr.csv')

  Region    Season Day_Type  4-hr  Hour_Tot  Load_Tot      Load_Avg
0   FRCC  shoulder  Weekday     1        12    223100  18591.666667
1   FRCC  shoulder  Weekday     2        12    290005  24167.083333
2   FRCC  shoulder  Weekday     3        12    329694  27474.500000
3   FRCC  shoulder  Weekday     4        12    342480  28540.000000
4   FRCC  shoulder  Weekday     5        12    318975  26581.250000
number of rows in dataset = 1044

     Region R_Group R_Subgroup    Season  Month  DOY  Hour   Load  Day  Year  \
300    FRCC    FRCC       FRCC    winter      1    2     2  15694    2  2011   
4034   FRCC    FRCC       FRCC  shoulder      3   74     1  15738   15  2011   
288    FRCC    FRCC       FRCC    winter      1    2     1  15750    2  2011   
5486   FRCC    FRCC       FRCC  shoulder      3   79     2  15822   20  2011   
3746   FRCC    FRCC       FRCC  shoulder      3   73     1  15831   14  2011   

           Date  DOW Day_Type  Hour_Counter  4-hr  Hour_Tot  Load_Tot  \
300 